In [1]:
import pandas as pd
import requests
import bs4
import re
import os

In [2]:
mainUrl = "https://www.calspia.go.kr/io/openapi/gid/selectOpenApiSvcListView.do"

In [3]:
mainresp = requests.get(mainUrl)

mainhtml = mainresp.text

In [4]:
mainbs = bs4.BeautifulSoup(mainhtml,"html.parser")

In [5]:
tableTag = mainbs.find(name="table", attrs={"id":"cmTable"})

In [6]:
inputTags = tableTag.findAll(name="input")

In [7]:
cmList = []

In [8]:
for i in range(0, len(inputTags)):
    cmValue = inputTags[i].attrs["value"]
    cmList.append(cmValue)

In [9]:
preFixUrl = "https://www.calspia.go.kr/io/openapi/gid/selectOpenApiSvcDtlView.do?apiSvcCd="

In [10]:
tableList = []

In [11]:
for cm in range(0, len(cmList)):

    url = preFixUrl+cmList[cm]

    resp = requests.get(url)

    html = resp.text

    bs = bs4.BeautifulSoup(html,"html.parser")

    metadata = pd.read_excel("../input/datalake_meta22.xlsx")

    ### 기본정보 설정 및 파라미터 설정 ###
    SITENAME = "건설사업정보시스템"
    DATANAME = bs.find(name="h2", attrs={"class":"subnav_tit_lv1"}).text
    print(DATANAME)
    APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

    targetData = metadata.loc[metadata.자료명==DATANAME]
    # URL = targetData["URL"].values[0]
    # SERVICENAME = targetData["서비스키"].values[0]
    # SERVICENAME = SERVICENAME.split(".")[0]
    # REQPARAM = targetData["요청변수"].values[0]
    # REQPARAM = REQPARAM.split(",")
    # PRIMARYKEY = targetData["기본키"].values[0]

    test = bs.find(name="table", attrs={"class":"common_tbl", 
                                        "id":"prnItemTable"} )

    tbodyTag = bs.find(name="tbody", attrs={"id":"prnItemTableBody"} )

    columnList = []
    rowList = []

    tableRows = tbodyTag.findAll(name="tr")

    tableRowsLen = len(tableRows)

    dataString1 = DATANAME.split(" ")[0]
    dataString2 = DATANAME.split(" ")[1]
    if cmList[cm]== "CM_0000331":
        dataString1="공사현황"
    if cmList[cm]=="FM_0000332":
        dataString1="시설물"
    if (cm >=25) & (cm <=37):
        dataString1="보상"
    if cmList[cm]=="LC_0000145":
        dataString1="환매권"
    if cmList[cm]=="LC_0000146":
        dataString1="부동산소유사실"
    if cmList[cm]=="PT_0000190":
        dataString1="검토조직"
    if cmList[cm]=="PT_0000321":
        dataString1="바디"
    skipAct=False
    for row in range(0, tableRowsLen):

        searchString = re.sub(r"[\n\t\s\r]*", "", tableRows[row].text)

        if  (searchString.count(dataString1) >=1)|(skipAct)|(searchString.count(dataString2) >=1):
            skipAct=True
        else:
            continue

        eachRow = tableRows[row]

        columns = eachRow.findAll(name="td")

        for i in range(0, len(columns)):

            columnValue = columns[i].text
            columnValue = re.sub(r"[\n\t\s\r]*", "", columnValue)
            columnList.append(columnValue)

        rowList.append(columnList)

        columnList = []

    columnNames = ["컬럼명","컬럼타입","컬럼정보"]

    tableDefine = pd.DataFrame( rowList[:],columns = columnNames)

    tableList.append(tableDefine)

공사정보 목록
기성보고 현황
품질계획 현황
안전관리비 현황
업체 목록
업체별 참여공사 목록
월간공정 현황
공사중인 시설물(터널) 목록
연도별 공사계약 목록
공사중인 시설물(절개사면) 목록
공사중인 시설물(통로박스) 목록
공사중인 시설물(옹벽) 목록
공사중인 시설물(교량) 목록
공사중인 시설물(수문) 목록
공사중인 시설물(제방) 목록
설계변경 내역서 현황
설계변경 현황
도로, 하천 공사정보
시설물 목록
시설물 기본제원
시설물 점검진단계획 목록
시설물 점검진단이력 목록
시설물 보수보강이력 목록
시설물 유지보수대장 목록
도로시설물 관리현황 정보
공공용지 취득실적-전체보상액 규모 기준 실적 현황
공공용지 취득실적-사업별 면적 및 보상액 기준 실적 현황
공공용지 취득실적-시도별 면적 및 보상액 기준 실적 현황
공공용지 취득실적 (채권보상실적)
공공용지 취득실적 (잔여지 및 간접보상)
공공용지 취득실적 (환매권 행사)
공공용지 취득실적 (부동산소유사실 확인서)
공공용지 취득실적 (대토보상)
공공용지 취득실적-기관별 취득면적 및 보상액 기준 실적 현황
공공용지 취득실적-취득방법별 면적 및 보상액 기준 실적 현황
공공용지 취득실적 (지목별 실적 현황)
공공용지 취득실적-공공사업의 물건보상액 기준 실적 현황
공공용지 취득실적-소속 기관별 취득면적 및 보상액 기준 실적 현황
도로점용허가내용 목록
품질검사성적서 등록 목록
품질검사성적서 상세정보 현황
품질검사전문기관 목록
품질검사전문기관 상세정보
건설자재 품질검사 등록정보
설계VE 상세 검토조직 조회
설계VE 상세 VE제안목록
설계VE 목록 조회
설계VE 상세정보
건설공사 사후평가 목록 조회
사후평가 수행실적표 - 사업수행성과 평가표
사후평가 수행실적표 - 사업효율 및 파급효과 평가표
설계도서등록 사업목록 조회
설계도서등록 상세정보 조회
건설공사 원가절감사례 목록 조회
건설공사 원가절감사례 상세 조회
설계VE 계약정보


In [12]:
for i in range(0, len(tableList)):
    print(cmList[i])
    sheetName = "sheet"
    try: 
        sheetName = tableList[i].loc[0].컬럼정보
    except:
        sheetName = "sheet"
    sheetName = sheetName.replace("/","_")
    print(sheetName)
    df = tableList[i].loc[1:]
    ## 파일이 존재하지 않으면 생성
    ## 파일이 존재하지 않으면 생성
    try:
        if not os.path.exists("output.xlsx"):
            with pd.ExcelWriter("output.xlsx", mode="w", engine="openpyxl") as writer:
                df.to_excel(writer, index=False,sheet_name = sheetName)

        else:
            with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl",if_sheet_exists="new") as writer:
                df.to_excel(writer, index=False,sheet_name = sheetName)  
    except Exception as e:
        print(e,i)

CM_0000080
공사정보목록
CM_0000109
기성보고조회
CM_0000110
특정공사,보고연월별품질계획
CM_0000111
안전관리비조회
CM_0000112
업체목록조회
CM_0000120
참여공사(전체공사)목록
CM_0000106
월간공정목록
CM_0000100
시설물(터널)목록
CM_0000082
공사계약현황
CM_0000101
시설물(절개사면)목록
CM_0000102
시설물(통로박스)목록
CM_0000103
시설물(옹벽)목록
CM_0000090
시설물(교량)목록
CM_0000104
시설물(수문)목록
CM_0000105
시설물(제방)목록
CM_0000108
설계변경내역서
CM_0000107
설계변경현황
CM_0000331
공사현황목록
FM_0000050
목록
FM_0000070
시설물종류코드(코드집참고)
FM_0000071
시설물종류코드(코드집참고)
FM_0000072
시설물번호
FM_0000073
시설물종류코드(코드집참고)
FM_0000074
유지보수대장목록
FM_0000332
시설물현황목록
LC_0000130
전체보상액규모
LC_0000131
사업별면적및보상액
LC_0000141
시_도별면적및보상액
LC_0000143
채권보상실적현황
LC_0000144
잔여지및간접보상실적현황
LC_0000145
환매권행사실적현황
LC_0000146
부동산소유사실확인서현황
LC_0000148
대토보상실적현황
LC_0000132
기관별취득면적및보상액
LC_0000140
취득방법별면적및보상액
LC_0000142
기준년도지목중전의보상액
LC_0000150
공공사업의물건보상액
LC_0000133
소속기관별취득면적및보상액
PM_0000160
도로점용허가내용목록
PM_0000210
품질검사성적서목록
PM_0000211
품질검사성적서상세정보
PM_0000310
품질검사전문기관목록
PM_0000311
품질검사전문기관상세정보
PM_0000333
품질검사기관
PT_0000190
검토조직조회
PT_0000200
설계VE평가채택여부
PT_0000170
설계VE목록
PT_0000172
